In [29]:
import numpy as np
import MSGtools as mst

/vols/talos/local/anaconda2-5.0.0/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [30]:
slon,slat = mst.get_msg_lon_lat('eu')

In [4]:
lat=np.linspace(15,30,61)
lon=np.linspace(91,102,45)

In [31]:
def find_index(x,y):
    xi=np.searchsorted(slat,x)
    yi=np.searchsorted(slon,y)
    return xi,yi

In [32]:
thisLat, thisLon = find_index(16.3,101.6)
print thisLat, thisLon

ValueError: object too deep for desired array

In [12]:
print lat[thisLat], lon[thisLon]

16.5 101.75


In [ ]:
def get_data(lat_input, long_input):
    lat_index  = np.nanargmin((lat-lat_input)**2)
    long_index = np.nanargmin((long-long_input)**2)
    return data[long_index][lat_index]

In [48]:
def get_data(lat_input, lon_input,lat_grid, lon_grid):
    lon_shape = lon_grid.shape
    lat_shape = lat_grid.shape
    lat_index  = np.nanargmin((lat_grid.ravel()-lat_input)**2)
    lon_index = np.nanargmin((lon_grid.ravel()-lon_input)**2)
    return np.unravel_index(lat_index, lat_shape), np.unravel_index(lon_index,lon_shape)

In [38]:
lon1 = np.array([[11,11.5,12],[11.1,11.6,12.1],[11.2,11.7,12.2]])
print lon1

[[11.  11.5 12. ]
 [11.1 11.6 12.1]
 [11.2 11.7 12.2]]


In [39]:
lat1 = np.array([[5,5.1,5.2],[4.5,4.6,4.7],[4,4.1,4.2]])
print lat1

[[5.  5.1 5.2]
 [4.5 4.6 4.7]
 [4.  4.1 4.2]]


In [49]:
thisLat2, thisLon2 = get_data(51,15,slat,slon)

In [50]:
print thisLat2, thisLon2

(167, 357) (596, 667)


In [54]:
print slat[thisLat2], slon[thisLon2]

51.000046 14.999965


In [56]:
np.nanargmin((slat.ravel()-51)**2)

133957

In [57]:
np.nanargmin((slon.ravel()-15)**2)

477467

In [59]:
slat.ravel()[133957]

51.000046

In [60]:
slon.ravel()[477467]

14.999965

In [27]:
import pyresample
import numpy as np

# Define lat-lon grid
lon = np.linspace(30, 40, 100)
lat = np.linspace(10, 20, 100)
lon_grid, lat_grid = np.meshgrid(lon, lat)
grid = pyresample.geometry.GridDefinition(lats=lat_grid, lons=lon_grid)

# Generate some random data on the grid
data_grid = np.random.rand(lon_grid.shape[0], lon_grid.shape[1])

# Define some sample points
my_lons = np.array([34.5, 36.5, 38.5])
my_lats = np.array([12.0, 14.0, 16.0])
swath = pyresample.geometry.SwathDefinition(lons=my_lons, lats=my_lats)

# Determine nearest (w.r.t. great circle distance) neighbour in the grid.
_, _, index_array, distance_array = pyresample.kd_tree.get_neighbour_info(
    source_geo_def=grid, target_geo_def=swath, radius_of_influence=50000,
    neighbours=1)

# get_neighbour_info() returns indices in the flattened lat/lon grid. Compute
# the 2D grid indices:
index_array_2d = np.unravel_index(index_array, grid.shape)

print "Indices of nearest neighbours:", index_array_2d
print "Longitude of nearest neighbours:", lon_grid[index_array_2d]
print "Latitude of nearest neighbours:", lat_grid[index_array_2d]
print "Great Circle Distance:", distance_array

Indices of nearest neighbours: (array([20, 40, 59]), array([45, 64, 84]))
Longitude of nearest neighbours: [34.54545455 36.46464646 38.48484848]
Latitude of nearest neighbours: [12.02020202 14.04040404 15.95959596]
Great Circle Distance: [5430.10966231 5893.32976544 4775.75963908]


In [28]:
from scipy.interpolate import RegularGridInterpolator

itp = RegularGridInterpolator( (lat, lon), data, method='nearest') 
res = itp(some_new_point)

NameError: name 'data' is not defined

In [61]:
combined_x_y_arrays = numpy.dstack([slat.ravel(),slon.ravel()])[0]

In [86]:
points_list = list(np.array([[51,52],[15,16]]).transpose())

In [87]:
print points_list

[array([51, 15]), array([52, 16])]


In [69]:
from scipy.spatial import cKDTree
import time

In [67]:
def do_kdtree(combined_x_y_arrays,points):
    mytree = cKDTree(combined_x_y_arrays)
    dist, indexes = mytree.query(points)
    return indexes

In [111]:
-

In [88]:
start = time.time()
results2 = do_kdtree(combined_x_y_arrays,points_list)
end = time.time()
print 'Completed in: ',end-start

Completed in:  0.923803091049


In [103]:
combined_coordinate_arrays = numpy.dstack([slat.ravel(),slon.ravel()])[0]

In [106]:
points = np.array([[10,11,12,13],[41.2,51.6,45,54]])
points_list = list(points.transpose())
print points_list

[array([10. , 41.2]), array([11. , 51.6]), array([12., 45.]), array([13., 54.])]


In [107]:
tree = cKDTree(combined_coordinate_arrays)

In [109]:
dist,indices = tree.query(points_list)

In [110]:
indices

array([479999, 479999, 479999, 479999])

In [112]:
results2 = get_index_kdtree(np.array([[41.2,51.6,45,54],[10,11,12,13]]),slon,slat)

In [113]:
print results2

[[371 513]
 [155 532]
 [285 562]
 [115 571]]


In [122]:
for i,res in enumerate(results2):
    print i
    print res
    print slat[res[0],res[1]], slon[res[0],res[1]]

0
[371 513]
41.2143 9.985558
1
[155 532]
51.609 10.979449
2
[285 562]
45.01125 11.983555
3
[115 571]
53.9748 12.986927


In [118]:
slat[371,513]

41.2143

In [76]:
from analysis_tools import grid_and_interpolation as gi

In [77]:
start = time.time()
results3 = gi.get_index((15,51),slon,slat)
end = time.time()
print 'Completed in: ',end-start

Completed in:  1.95772409439


In [78]:
print results3

(array(167), array(621))


In [128]:
print(u'\N{GREEK CAPITAL LETTER THETA}r\N{SUBSCRIPT TWO}')

Θr₂
